# Prepare Experiment & Deepspeed config (**MANDATORY**)
***

In [4]:
ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 24,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
        "offload_optimizer": {
            "device": "none",
        },
        "offload_params": {
            "device": "none"
        },
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 200,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

training_args = {
    "num_train_epochs": 5,
    "gradient_accumulation_steps": 1,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 256,
    "fp16": True,
    "weight_decay": 0.0,
    "warmup_steps": 0,
    "learning_rate": 1e-5,
    "logging_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 1,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_BLEU-4",
    "greater_is_better": True,
}

num_gpus = 1
model_name ="t5-small"
logdir = "data/models/t5-small/rot-generator/"
override_logdir = True
block_size = 100


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
from datasets import load_dataset
import time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import datasets
from tqdm import tqdm
import nltk

# Tokenize the dataset
***

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=block_size)

NameError: name 'model_name' is not defined

In [6]:
from sklearn.model_selection import train_test_split

def load_social_chem101():
    a = pd.read_csv("data/social-chem-101/social-chem-101.v1.0.tsv", sep="\t")
    return a

social_chem = load_social_chem101()
#social_chem = social_chem[social_chem["split"] == "train"]
social_chem = social_chem.dropna(subset=["rot-categorization", "rot-judgment", "action", "rot-agree"])
social_chem = social_chem[social_chem["rot-agree"] >= 3.0]
social_chem = social_chem[social_chem["rot-bad"] == 0]
social_chem = social_chem[social_chem["rot-categorization"].apply(lambda x: "morality-ethics" in x or "social-norms" in x)]
social_chem = social_chem[social_chem["rot-judgment"].apply(lambda x: "{" not in x)]
social_chem = social_chem[social_chem.apply(lambda x: max(len(x["rot"]), len(x["action"]) + len(x["rot-judgment"])) <= block_size, axis=1)]
social_chem = social_chem[["action", "rot-judgment", "rot"]].groupby("rot", as_index=False).nth(0)

train, dev = train_test_split(social_chem, test_size=0.2, random_state=42)
dev, test = train_test_split(dev, test_size=0.5, random_state=42)

dataset = datasets.DatasetDict()
dataset["train"] = datasets.Dataset.from_pandas(train)
dataset["dev"] = datasets.Dataset.from_pandas(dev)
dataset["test"] = datasets.Dataset.from_pandas(test)

In [ ]:
def tokenize_input(samples):
    inp = tokenizer(samples["rot-judgment"], samples["action"], truncation=True, padding="max_length", max_length=block_size)
    with tokenizer.as_target_tokenizer():
        out = tokenizer(samples["rot"], truncation=True, padding="max_length", max_length=block_size)
    inp["labels"] = out["input_ids"]
    return inp

tokenized_data = dataset.map(tokenize_input, batched=True, batch_size=10000).shuffle()

# Load the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Prepare Trainer
***

In [ ]:
# Taken from Moral Stories Github:
# https://github.com/demelin/moral_stories/blob/master/experiments/utils.py

import sacrebleu
from sacrerouge.metrics import Rouge

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return compute_gen_metrics(decoded_preds, decoded_labels)

def compute_gen_metrics(preds, targets):
    """ Aggregates generation metrics. """
    assert len(preds) == len(targets)
    return {'BLEU-4': compute_bleu(preds, targets),
            'ROUGE-L': compute_rouge(preds, targets)}    

def compute_bleu(preds, targets):
    """ Computes corpus-level BLEU for the generated sequences. """
    targets = [targets]
    bleu = sacrebleu.corpus_bleu(preds, targets)
    return bleu.score

def compute_rouge(preds, targets):
    """ Computes ROUGE-L for the generated sequences. """
    rouge = Rouge(compute_rouge_l=True)
    rouge_out = rouge.evaluate(preds, [[tgt] for tgt in targets])
    return rouge_out[0]['rouge-l']['f1']

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

import torch

data_collator = DataCollatorForSeq2Seq(tokenizer, model)

training_args = Seq2SeqTrainingArguments(
    output_dir=logdir,
    overwrite_output_dir=override_logdir,
    logging_dir=logdir,
    deepspeed= logdir + "/ds_config.json",
    report_to="tensorboard",
    predict_with_generate=True,
    **training_args
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_data["test"], metric_key_prefix="test")